# Device Setting

## import

In [ ]:
# Import necessary packages
import numpy as np
import os
import pandas as pd
# Import for read data function
from scipy import io
from scipy.io import loadmat
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.nn.functional as F

import random

## connect to dirve

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## set GPU

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# remove

In [ ]:
# os.chdir('/content')
# !ls
# !rm -rf test
# !ls
# !mkdir test
# !ls

# Model Setting

## model structure

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Model(nn.Module):
    def __init__(self,numNeurons=16):
        super(Model, self).__init__()

        # self.conv1 = nn.Conv2d(1,3,3,stride=2,padding=0)
        # self.bn2d1 = nn.BatchNorm2d(3)

        self.fc1 = nn.Linear(160, numNeurons*8).double()
        self.fc2 = nn.Linear(numNeurons*8, numNeurons*8).double()
        self.fc3 = nn.Linear(numNeurons*8, numNeurons*4).double()
        self.fc4 = nn.Linear(numNeurons*4, numNeurons*2).double()
        self.fc5 = nn.Linear(numNeurons*2, 4).double()
        self.dropout=nn.Dropout(p=0.25)

    def forward(self, x):
        # x = self.conv1(x)
        # x=self.bn2d1(x)
        # x=F.relu6(x)

        # x=x.view(x.size()[0],-1) # flatten the activation
        x=self.fc1(x)
        x=F.relu(x)
        x = self.dropout(x)

        x=self.fc2(x)
        x=F.relu(x)
        x = self.dropout(x)

        x=self.fc3(x)
        x=F.relu(x)

        x=self.fc4(x)
        x=F.relu(x)

        x=self.fc5(x)

        return x

# Data Preprocessing

## load .mat file

In [ ]:
path = "/content/drive/MyDrive/finetune_1129"

os.chdir(path)
files = os.listdir(path)
files = [ff for ff in files if '.mat' in ff]

def extract_number(filename):
    return int(''.join(filter(str.isdigit, filename)))

files = sorted(files, key=extract_number)

print(files)

['1s-01-Image(1-14).mat', '1s-02-Image(15-30).mat', '1s-03-Image(31-53).mat', '1s-04-Image(54-70).mat', '1s-05-Image(71-92).mat', '1s-06-Image(93-110).mat', '1s-07-Image(111-126).mat', '1s-08-Image(127-142).mat', '1s-09-Image(143-160).mat', '2s-01-Image(161-164).mat', '2s-02-Image(165-172).mat', '2s-03-Image(173-181).mat', '2s-04-Image(182-188).mat', '2s-05-Image(189-191).mat', '2s-06-Image(192-194).mat', '2s-07-Image(195-198).mat', '2s-08-Image(199-200).mat']


In [ ]:
Data = []

for i in files:
  f = loadmat(i)
  data = f['cellList']['meshData']
  cells_file = data[0][0][0][0][0]
  # print(len(cells_file))
  for j in range (len(cells_file)):
    Base = cells_file[j][0][0]
    Data.append(Base['signal1'])
    Data.append(Base['signal2'])

print(len(Data))
# print(Data[0])
# print(Data[1])

400


## data preprocessing

expand length to 160

In [ ]:
def extrapo(s, extra_length):
  desired_length = 80
  new_vector = np.zeros(desired_length)

  cnt = 0
  for i in range(len(s)-1):
    cnt = i
    if i == extra_length:
      break
    new_vector[2 * i] = s[i]
    new_vector[2 * i + 1] = ((s[i] + s[i+1]) / 2)

  cnt_s = cnt
  cnt = cnt * 2
  nokori_len = len(s) - cnt_s
  for i in range(nokori_len):
    new_vector[cnt] = s[cnt_s]
    cnt += 1
    cnt_s += 1

  for i in range(desired_length-cnt):
    new_vector[cnt+i] = s[cnt_s-1]

  return new_vector

In [ ]:
def interpo(s):
  desired_length = 80

  remove_length = len(s) - desired_length
  indices_to_remove = random.sample(range(len(s)), remove_length)

  new_vector = [value for index, value in enumerate(s) if index not in indices_to_remove]

  # print(len(new_vector))
  for i in range(len(new_vector)):
    new_vector[i] = new_vector[i]*2 / 2

  return new_vector

In [ ]:
modified_Data = []
additional_Data = []
indexes = []

for i in range(int(len(Data)/2)):
  temp1 = [element for sublist in Data[i*2] for element in sublist]
  temp1 = np.array(temp1)
  # normalize
  if max(temp1) != 0:
    temp1 = 255*temp1/max(temp1)
  # print(len(temp1))
  # temp3 = np.flip(temp1)
  if len(temp1) < 3:
    # indexes.append(i)
    print(i, "<3")
    continue
  if len(temp1) < 80:
    temp1 = extrapo(temp1, (80-len(temp1)))
    # temp3 = extrapo(temp3, (80-len(temp3)))
  if len(temp1) >= 80:
    temp1 = interpo(temp1)
    # print(temp1[0].dtype)

  temp2 = [element for sublist in Data[i*2+1] for element in sublist]
  temp2 = np.array(temp2)
  # normalize
  if max(temp2) != 0:
    temp2 = 255*temp2/max(temp2)
  # temp4 = np.flip(temp2)
  if len(temp2) < 80:
    temp2 = extrapo(temp2, (80-len(temp2)))
    # temp4 = extrapo(temp4, (80-len(temp4)))
  if len(temp2) >= 80:
    temp2 = interpo(temp2)
  temp5 = np.concatenate((temp1, temp2), axis = 0)
  modified_Data.append(temp5)

  # temp6 = np.concatenate((temp3, temp4), axis = 0)
  # additional_Data.append(temp6)

# print(modified_Data[4])
print(len(modified_Data))
print(indexes)
# print(modified_Data[1].dtype)

200
[]


exclude some bad data
- length < 3
- label = 6

In [ ]:
# for i in range(len(modified_Data)):
#   if labels[i] == 6:
#     indexes.append(i)

pruned_modified_Data = []
# modified_additional_Data = []
for i in range(len(modified_Data)):
  if i not in indexes:
    pruned_modified_Data.append(modified_Data[i])
# for i in range(len(additional_Data)):
#   if i not in indexes:
#     modified_additional_Data.append(additional_Data[i])
# pruned_labels = np.delete(labels, indexes)
# temp = np.copy(pruned_labels)
# pruned_labels = np.concatenate((pruned_labels, temp), axis=0)



print(len(indexes))
print(len(pruned_modified_Data))
# print(len(pruned_labels))
print(indexes)
# print(pruned_labels)
print(pruned_modified_Data[0])
print(pruned_modified_Data[0].dtype)

0
200
[]
[  0.           2.01026893   4.02053785   6.22972679   8.43891525
   8.39430809   8.34969997   6.54216194   4.73462343   5.44377232
   6.15292072   6.47055864   6.78819656  11.38145256  15.97470856
  10.73902798   5.5033474   12.82868671  20.15402603  18.53079796
  16.90756989  13.76337624  10.61918163  20.78191948  30.94465828
  29.85037231  28.75608635  28.0682869   27.38048935  26.08950424
  24.79851913  23.45892334  22.11932945  25.94384003  29.76834869
  26.17612839  22.58390999  32.56075287  42.53759384  39.71721649
  36.89684296  47.36993027  57.84301758  51.83301544  45.82301331
  57.70750809  69.59200287  69.90341187  70.21481323  83.55450439
  96.89419556  90.85340118  84.81260681  95.01677704 105.22094727
 108.11608124 111.01121521 102.79483032  94.57844543 119.84204102
 145.1056366  148.41462708 151.72361755 193.69422913 235.6648407
 232.61160278 229.55836487 255.         248.45753479 184.05410767
 100.05764771  85.74124908  32.21974564  42.29249954  31.03929901
  

## build dataset

In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if self.labels is not None:
          return self.data[idx], self.labels[idx]
        else:
          return self.data[idx]

create a dataset for testing

In [ ]:
batch_size = 1

test_data = pruned_modified_Data
test_data_tensor = [torch.from_numpy(sample) for sample in test_data]
test_set = MyDataset(test_data_tensor, None)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False)

print(test_data[1].dtype)

float64


# Evaluate

make prediction

In [ ]:
os.chdir("/content")
predict = []
model = Model().to(device)
model.load_state_dict(torch.load("drive/MyDrive/Model_5.pt"))
model.eval()
with torch.no_grad():
    for i, data in enumerate(test_loader):
        # print(i)
        # print(data)
        inputs = data
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability

        for y in test_pred.cpu().numpy():
            predict.append(y)

print(predict)

<ipython-input-14-641fd0191e8b>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("drive/MyDrive/Model_5.pt"))


[1, 3, 0, 2, 0, 1, 1, 3, 2, 1, 1, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 1, 3, 3, 1, 3, 3, 2, 0, 3, 2, 1, 3, 1, 1, 0, 1, 3, 2, 1, 2, 1, 3, 0, 0, 1, 1, 2, 0, 1, 3, 3, 1, 1, 3, 1, 1, 3, 2, 3, 0, 2, 1, 3, 3, 3, 1, 0, 3, 1, 1, 0, 0, 3, 0, 1, 1, 3, 2, 1, 1, 1, 2, 1, 1, 1, 0, 2, 3, 2, 2, 1, 0, 0, 0, 0, 1, 0, 2, 1, 0, 3, 2, 1, 0, 0, 3, 2, 0, 2, 2, 1, 0, 0, 1, 0, 0, 2, 1, 0, 0, 2, 1, 1, 1, 2, 3, 1, 1, 1, 0, 1, 2, 2, 1, 3, 1, 0, 3, 2, 3, 2, 0, 0, 3, 2, 2, 0, 2, 3, 3, 0, 2, 2, 2, 3, 0, 2, 3, 3, 2, 3, 2, 0, 0, 3, 3, 0, 0, 3, 0, 2, 3, 3, 2, 3, 2, 3, 2, 2, 2, 3, 2, 2, 3, 2, 3, 3, 2, 3, 2, 2, 3, 3, 2, 3, 3, 3, 3]


Write prediction to a CSV file

In [ ]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(predict):
        f.write('{},{}\n'.format(i, y))